In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer

# For BM25
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/adamwagnerhoegh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Generate retrieval corpus

In [9]:
path_adam = '/Users/adamwagnerhoegh/Documents/Legal data/domsdatabasen.retsinformation_newer.json'
path_asger = "/Users/asgerkromand/Library/CloudStorage/OneDrive-UniversityofCopenhagen/0. SDS/1 deep learning and nlp/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json"
path_andreas = '' #missing

# Define a function that can cycle through paths the above paths try them out, and yield the path
def path():
    paths = cycle([path_adam, path_asger, path_andreas])
    for path in paths:
        if path != '':
            try:
                with open(path, 'r') as f:
                    data = json.load(f)
                return data
            except:
                pass
        else:
            raise FileNotFoundError('No path to data found')

retsinfo = path()
    
rag_list = []
idx = 0
for lov in tqdm(retsinfo):
    for kapitel in lov['kapitler']:
        lov_navn = lov['shortName']
        for paragraffer in kapitel['paragraffer']:
            temp_paragraf_dict = {}
            temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
            temp_paragraf_dict['lovnavn'] = lov_navn
            temp_paragraf_list = []
            for styk in paragraffer['stk']:
                temp_paragraf_list.append(styk['tekst'])
            temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
            rag_list.append(temp_paragraf_dict)

with open("rag_list.txt", "w") as file:
    for item in rag_list:
        file.write(f"{item}\n")

100%|██████████| 1637/1637 [00:00<00:00, 10544.75it/s]


## Generate dev set

In [10]:
# load excel files in dev set folder
import os

dev_set_folder = "devset"

dfs = []
for file in os.listdir(dev_set_folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(dev_set_folder, file))
        dfs.append(df)

# merge all excel
dev_set = pd.concat(dfs, ignore_index=True)

# add csv
rag_batch_1_with_qa = pd.read_csv("devset/rag_batch_1_with_qa.csv", sep=",").iloc[:, 1:].dropna()
rag_batch_1_with_qa.columns = dev_set.columns
dev_set = pd.concat([dev_set, rag_batch_1_with_qa], ignore_index=True)

# output dev set
dev_set.to_csv("devset/dev_set.csv", index=False)

## Vectorize retrieval corpus

### Sparse retrieval TF-IDF

In [11]:
rag_list2 = rag_list

def preprocess(rag_list):
    # extract and preprocess text
    corpus = [item['text'] for item in rag_list]
    corpus = [re.sub('\\s{2,}', ' ', 
                     re.sub('\\W|[0-9]|§', ' ',
                           item.lower())) for item in corpus]

    # remove stopwords
    #nltk.download('punkt')
    stop_words = set(stopwords.words('danish'))
    corpus = [' '.join(word for word in text.split() 
                      if word not in stop_words) for text in tqdm(corpus)]
    
    return corpus

corpus = preprocess(rag_list2)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

100%|██████████| 42593/42593 [00:00<00:00, 115199.02it/s]


### Dense retrieval

In [12]:
## WRITE LATER

## RAG retriever

### Sparse retrieval pipeline for BM25


In [13]:
def sparse_retrieval(question, sparse_matrix, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """

    # preprocess and vectorize question
    question_processed = [re.sub('\\s{2,}', ' ', 
                               re.sub('\\W|[0-9]|§', ' ',
                                     question.lower()))]
    
    # remove stopwords
    stop_words = set(stopwords.words('danish'))
    question_processed = [' '.join(word for word in text.split() 
                                 if word not in stop_words) for text in question_processed]
    
    question_vector = vectorizer.transform(question_processed)

    # sparse retrieval (cosine similarity)
    sparse_retrieval = sparse_matrix.dot(question_vector.T).toarray()

    # get top k paragraphs
    top_k_indices = np.argsort(sparse_retrieval.flatten())[-k:]

    return top_k_indices

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k_indices = sparse_retrieval(random_question, X)
for i in top_k_indices:
    print(f'{rag_list2[i]["paragraf_nr"]}: {rag_list2[i]["text"]}')

Hvad sker der med en kapitalejers stemmeret på generalforsamlingen, hvis kapitalejeren har undladt rettidigt at efterkomme det centrale ledelsesorgans krav om indbetaling af udestående beløb på en kapitalandel? 

§ 170.: Det centrale ledelsesorgans beslutning i henhold til § 169 skal i sin helhed optages i vedtægterne.
§ 52.: Ejerbogen for et kapitalselskab, som har udstedt navnekapitalandele, skal indeholde følgende oplysninger, jf. dog stk. 3: Kapitalejers samlede beholdning af kapitalandele. Kapitalejers samlede beholdning af kapitalandele. Kapitalejers og panthavers navn og bopæl og for virksomheder navn, cvr-nummer og hjemsted, jf. stk. 2. Dato for erhvervelse, afhændelse eller pantsætning, herunder kapitalandelenes størrelse. De stemmerettigheder, der er knyttet til kapitalandelene. Er kapitalejeren eller panthaveren en udenlandsk statsborger eller en udenlandsk juridisk person, skal meddelelsen, jf. § 53, stk. 1, vedlægges anden dokumentation, der sikrer en entydig identifikatio

In [14]:
# def bm25_retrieval(question, bm25_model, rag_list, k=3):
#     """
#     Function that takes a question and returns a list of the most relevant paragraphs based on BM25.
#     """
#     # Preprocess and tokenize the question
#     question_processed = re.sub(r'\s{2,}', ' ', 
#                                  re.sub(r'\W|[0-9]|§', ' ', question.lower()))
#     stop_words = set(stopwords.words('danish'))
#     question_tokens = [word for word in question_processed.split() if word not in stop_words]

#     # Get BM25 scores for the query
#     scores = bm25_model.get_scores(question_tokens)

#     # Get the top k results
#     top_k_indices = np.argsort(scores)[-k:][::-1]  # Sort scores in descending order

#     # Return the top k paragraphs
#     return [(rag_list[i]['paragraf_nr'], rag_list[i]['text'], scores[i]) for i in top_k_indices]

# # Example Usage
# random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
# print(f"Question: {random_question}\n")

# # Assuming bm25 is the initialized BM25Okapi model
# top_k_results = bm25_retrieval(random_question, bm25, rag_list2, k=3)

# # Print top-k results
# for paragraf_nr, text, score in top_k_results:
#     print(f"{paragraf_nr}: {text} (Score: {score:.2f})")

### Create embedding corpus

In [15]:

def create_embedding_matrix(pooling, save=False, save_folder=None):
    # initialise model
    bert_tokenizer = AutoTokenizer.from_pretrained("KennethTM/bert-base-uncased-danish")
    bert_model = AutoModel.from_pretrained("KennethTM/bert-base-uncased-danish")

    # define device
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

    # move model to device
    bert_model.to(device)

    # create list of embedding vectors to concatenate into a torch tensor
    embeddings = []

    # index to track numer of mistakes
    idx = 0

    for item in tqdm(rag_list):
        # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
        try:
            # tokenize texts
            input_ids = bert_tokenizer.encode(item['text'], return_tensors='pt').to(device)
            # run through BERT
            with torch.no_grad():  # disable gradient computation for inference
                outputs = bert_model(input_ids)
            
            # different kinds of pooling
            if pooling == 'cls':
                embedding_vector = outputs.last_hidden_state[:, 0, :]
            elif pooling == 'max':
                embedding_vector = torch.max(outputs, dim=1)[0]
            elif pooling == 'mean':
                embedding_vector = torch.mean(outputs, dim=1)
            else:
                raise ValueError(f"Unknown pooling method: {pooling}")
            
            # add cls-vector to list of embeddings
            embeddings.append(embedding_vector)
        except:
            # if error then count errors with this
            embeddings.append(torch.zeros(1, 768))
            idx += 1

    print(f'{idx} no. of errors')

    # concatenate list into torch tensor
    embeddings_tensor = torch.cat(embeddings, dim=0)

    if save == True:
        # make sure that folder exists
        os.makedirs(save_folder, exist_ok=True)

        # save tensor 
        torch.save(embeddings_tensor, f'{save_folder}/{pooling}_embeddings_tensor.pt')

    return embeddings_tensor



In [16]:
#create_embedding_matrix(pooling='cls')

### Dense retrieval pipeline

In [17]:
bert_tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
bert_model = AutoModel.from_pretrained("vesteinn/DanskBERT")

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# you can load cls or max respectively, mean still needs to be created
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

In [19]:
def dense_retrieval(question, pooling='cls', k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    pooling = 'cls', 'max' or 'mean'
    """
    
    # Encode the input sentence
    input_ids = bert_tokenizer.encode(question, return_tensors="pt")  # Encode and add batch dimension
    # Pass the input through the model
    
    with torch.no_grad():  # disable gradient computation for inference
        outputs = bert_model(input_ids)
    
    if pooling == 'cls':
        # Extract the CLS token representation
        embedding_vector = outputs.last_hidden_state[:, 0, :]
    
    elif pooling == 'max':
        embedding_vector = torch.max(outputs.last_hidden_state, dim=1)[0]

    elif pooling == 'mean':
        embedding_vector = torch.mean(outputs.last_hidden_state, dim=1)
    
    # normalise the cls-embedding and the embedding matrix so that the dot product
    # below is now cosine similarity
    embedding_vector_normalised = embedding_vector / torch.norm(embedding_vector, dim=-1, keepdim=True)
    embeddings_matrix_normalised = embeddings_matrix / torch.norm(embeddings_matrix, dim=-1, keepdim=True)

    # finding most similar vectors with dot product
    dense_retrieval = embeddings_matrix_normalised @ torch.transpose(embedding_vector_normalised, 0, 1)
    
    # sorting retrieved embeddings. this leaves a bunch of nan-values at the top
    sorted_retrieval = torch.sort(dense_retrieval, descending=True, stable=True, dim=0)

    # recreating list, sorting out nan-values
    fixed_retrieval_list = [(item, idx) for (item, idx) in zip(sorted_retrieval[0], sorted_retrieval[1]) if torch.isnan(item) == False]

    # get top k paragraphs
    top_k_indices = [item[1] for item in fixed_retrieval_list[:k]]

    return top_k_indices

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n') 
top_k = dense_retrieval(random_question, pooling='cls', k=3)
for i in top_k:
    print(f'{rag_list[i]["text"]}')

Hvad kan et jobcenter give et tilbud om, udover ansættelse med løntilskud, nytteindsats samt vejledning og opkvalificering? 

Jobcenteret kan give tilbud om følgende: Virksomhedspraktik, jf. kapitel 11. Virksomhedspraktik, jf. kapitel 11. Ansættelse med løntilskud, jf. kapitel 12. Nytteindsats, jf. kapitel 13. Vejledning og opkvalificering, jf. kapitel 14. Tilbud kan gives hver for sig eller i kombination.
Forud for ansættelse med løntilskud skal spørgsmålet om ansættelsen have været drøftet mellem arbejdsgiveren og en repræsentant for de ansatte hos arbejdsgiveren.
Det er en betingelse for retten til jobpræmie, at personen er registreret i CPR-registeret med bopæl i Danmark.


### Evaluation

In [20]:
# This function is a bit broken, don't think we should use it

# def evaluate(model_name, retrieval_method, metric, k):
#     """
#     model_name = 'KennethTM/gpt-neo-1.3B-danish' or 'strombergnlp/dant5-large'
#     retrieval_method = 'tf-idf', 'bm25' or 'dense'
#     metric = 'bleu', 'rouge' or 'meteor'
#     """
#     # set device to mps
#     device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

#     # load AutoTokenizer for model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     # generate list of answers to fill
#     answers = []

#     # load neo
#     # the loops are made for each model to not waste compute on loading models for each question

#     if model_name == 'KennethTM/gpt-neo-1.3B-danish':
#         model = AutoModelForCausalLM.from_pretrained("KennethTM/gpt-neo-1.3B-danish").to(device)
    
#         for question in tqdm(dev_set['question, str'], desc='Answering questions with neo'):

#             if retrieval_method == 'tf-idf':
#                 # create list of paragraphs by getting indexes for best hits with sparse_retrieval
#                 paragraphs = [rag_list[i]['text'] for i in sparse_retrieval(question, X, k=3)]
#                 # join list into long string
#                 paragraphs = ' '.join(paragraphs)

#             elif retrieval_method == 'bm25':
#                 paragraphs = bm25_retrieval(rag_list=rag_list, k=3)
            
#             elif retrieval_method == 'dense_retrieval':
#                 # create list of paragraphs by getting indexes for best hits with sparse_retrieval
#                 paragraphs = [rag_list[i]['text'] for i in dense_retrieval(question, k=3)]
#                 # join list into long string
#                 paragraphs = ' '.join(paragraphs)

#             # assemble all in prompt
#             prompt = f'Kontekst: {paragraphs} Spørgsmål: {question} Svar: '

#             # tokenize
#             input_ids = tokenizer(prompt, return_tensors="pt").to(device)

#             # set max_length to no. of tokens in prompt + 100 (the 100 are thus for generation)
#             max_length = int(input_ids['input_ids'].size(1)) + 100

#             # generate answer with no_grad() to save compute
#             with torch.no_grad():
#                 outputs = model.generate(
#                     input_ids,
#                     max_length=max_length,
#                     pad_token_id=tokenizer.eos_token_id)
            
#             # decode the generated answer
#             answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

#             answers.append(answer)
    

#     # load T5
#     elif model_name == 'strombergnlp/dant5-large':
#         model = T5ForConditionalGeneration.from_pretrained("strombergnlp/dant5-large").to(device)

#         for question in tqdm(dev_set['question, str'], desc='Answering questions with T5'):

#             if retrieval_method == 'tf-idf':
#                 # create list of paragraphs by getting indexes for best hits with sparse_retrieval
#                 paragraphs = [rag_list[i]['text'] for i in sparse_retrieval(question, X, k=3)]
#                 # join list into long string
#                 paragraphs = ' '.join(paragraphs)

#             elif retrieval_method == 'bm25':
#                 paragraphs = bm25_retrieval(rag_list=rag_list, k=3)
            
#             elif retrieval_method == 'dense_retrieval':
#                 # create list of paragraphs by getting indexes for best hits with sparse_retrieval
#                 paragraphs = [rag_list[i]['text'] for i in dense_retrieval(question, k=3)]
#                 # join list into long string
#                 paragraphs = ' '.join(paragraphs)

#             # assemble all in input
#             input_text = f"Spørgsmål: {question} Kontekst: {paragraphs} Svar:"

#             # tokenize
#             input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

#             # generate answer with no_grad() to save compute
#             with torch.no_grad():
#                 outputs = model.generate(
#                     input_ids,
#                     max_length=max_length,
#                     pad_token_id=tokenizer.eos_token_id)
            
#             with torch.no_grad():
#                 outputs = model.generate(input_ids, max_length=100)

#             # Decode and print the generated answer
#             answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

#             answers.append(answer)
        
#     # choosing metric to evaluate answers
#     if metric == 'bleu':
#         scores = []
#         idx = 0

#         for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with bleu'):
#             try:
#                 scores.append(nltk.translate.bleu_score.sentence_bleu([true_answer], pred_answer))
#             except:
#                 print(f'Error when computing bleu-score at index {idx}')
#             idx += 1
    
#     elif metric == 'meteor':
#         scores = []
#         idx = 0

#         for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with meteor'):
#             try:
#                 scores.append(nltk.tranlsate.meteor_score([true_answer], pred_answer))
#             except:
#                 print(f'Error when computing meteor-score at index {idx}')
#             idx += 1

#     elif metric == 'rouge':
#         scores = []
#         idx = 0

#         scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

#         for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with rouge'):
#             try:
#                 scores.append(score.score(true_answer, pred_answer)['rouge1'])
#             except:
#                 print(f'Error when computing meteor-score at index {idx}')
#             idx += 1

#     print(f'{metric}-scores for {model_name} using {retrieval_method}: {np.mean(scores)}')
        

### Create columns for retrieved documents

In [ ]:
# looks like the dense retrievers are generally retrieving shorter paragraphs

tf_idf_k1 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=1'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in sparse_retrieval(question, X, k=1)])
    tf_idf_k1.append(documents)

tf_idf_k2 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=2'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in sparse_retrieval(question, X, k=2)])
    tf_idf_k2.append(documents)

tf_idf_k3 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=3'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in sparse_retrieval(question, X, k=3)])
    tf_idf_k3.append(documents)

# insert BM25

dense_cls_k1 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=1'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='cls', k=1)])
    dense_cls_k1.append(documents)

dense_cls_k2 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=2'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='cls', k=2)])
    dense_cls_k2.append(documents)

dense_cls_k3 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=3'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='cls', k=3)])
    dense_cls_k3.append(documents)

dense_max_k1 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=1'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='max', k=1)])
    dense_max_k1.append(documents)

dense_max_k2 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=2'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='max', k=2)])
    dense_max_k2.append(documents)

dense_max_k3 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=3'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in dense_retrieval(question, pooling='max', k=3)])
    dense_max_k3.append(documents)


Dense CLS, k=3: 100%|██████████| 106/106 [00:28<00:00,  3.78it/s]


In [36]:
# adding to dev set and saving locally
dev_set['tf_idf_k1'] = tf_idf_k1
dev_set['tf_idf_k2'] = tf_idf_k2
dev_set['tf_idf_k3'] = tf_idf_k3
dev_set['dense_cls_k1'] = dense_cls_k1
dev_set['dense_cls_k2'] = dense_cls_k2
dev_set['dense_cls_k3'] = dense_cls_k3
dev_set['dense_max_k1'] = dense_max_k1
dev_set['dense_max_k2'] = dense_max_k2
dev_set['dense_max_k3'] = dense_max_k3

dev_set.to_csv("devset/dev_set.csv", index=False)

### Evaluating neo

In [82]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
MODEL_NAME = "KennethTM/gpt-neo-1.3B-danish"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set the device
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(DEVICE)


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [124]:
neo_answers_k1 = []
# evaluating tf-idf
for question in tqdm(dev_set['question, str'], desc='Answering questions'):
    # prepended documents:
    documents = ' '.join([rag_list[i]['text'] for i in sparse_retrieval(question, X, k=1)])

    # assemble a prompt from the documents, question and prompting an answer
    prompt = f"Kontekst: {documents} Spørgsmål: {question}\nSvar:"

    # tokenize
    input_ids = tokenizer(prompt, padding=True, return_tensors="pt").input_ids.to(DEVICE)

    max_length = len(input_ids[0]) + 100

    # generate an answer within torch.no_grad() to save compute
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            # generation set to stop at '.' as it otherwise just repeats itself (think it's because we don't sample)
            eos_token_id=tokenizer.convert_tokens_to_ids(".")
        )

    # decode generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # append answer to list
    neo_answers_k1.append(answer[len(prompt):].strip())  # strip the prompt to leave just the answer

Answering questions: 100%|██████████| 106/106 [13:33<00:00,  7.67s/it]


In [127]:
dev_set['neo_answers_k1'] = neo_answers_k1

In [ ]:
tf_idf